# Craigslist Used Cars — Data Cleaning Pipeline

**Runtime:** GPU → T4  
**Dataset:** `vehicles.csv` (~426k rows, 26 columns)  
**Output:** `cleaned_cars.csv` + `summary_stats` dict

---
### Steps
1. Mount Google Drive & upload raw CSV  
2. Load & baseline inspection  
3. Drop price outliers (`< 500` or `> 150 000`)  
4. Drop odometer outliers (`> 300 000`)  
5. Parse `posting_date` → `year_month`, `month`, `day_of_week`  
6. Keep only top-20 makes by listing count  
7. Add `price_per_mile` feature  
8. Summary stats & save  

In [1]:
# ── Verify T4 GPU is attached (informational only — cleaning is CPU-bound) ──
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total',
                         '--format=csv,noheader'], capture_output=True, text=True)
print('GPU:', result.stdout.strip() if result.returncode == 0 else 'No GPU detected — CPU mode')

GPU: Tesla T4, 15360 MiB


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

print('pandas', pd.__version__, '| numpy', np.__version__)

pandas 2.2.2 | numpy 2.0.2


## 1 — Mount Google Drive & locate the CSV

Upload `vehicles.csv` to your Drive at **`My Drive/car_data/vehicles.csv`**  
(or adjust `RAW_PATH` below to match wherever you put it).

In [3]:
from google.colab import drive, files
import os

drive.mount('/content/drive', force_remount=False)

# ── Adjust this path to match where you placed vehicles.csv in Drive ──
RAW_PATH    = '/content/drive/MyDrive/car_data/vehicles.csv'
OUTPUT_PATH = '/content/drive/MyDrive/car_data/cleaned_cars.csv'

# Create the folder if it doesn't exist
os.makedirs(os.path.dirname(RAW_PATH), exist_ok=True)

if os.path.exists(RAW_PATH):
    print(f'Found:  {RAW_PATH}')
    print(f'Output: {OUTPUT_PATH}')
else:
    print('vehicles.csv not found at the expected path.')
    print('Launching file-picker so you can upload it now...')
    uploaded = files.upload()          # browser dialog
    fname = list(uploaded.keys())[0]
    import shutil
    shutil.move(fname, RAW_PATH)
    print(f'Saved to {RAW_PATH}')

Mounted at /content/drive
Found:  /content/drive/MyDrive/car_data/vehicles.csv
Output: /content/drive/MyDrive/car_data/cleaned_cars.csv


## 2 — Load & baseline inspection

In [4]:
# ── dtype hints speed up parsing of the large file ──
DTYPES = {
    'id':           'Int64',
    'price':        'float64',
    'year':         'float64',
    'odometer':     'float64',
    'lat':          'float64',
    'long':         'float64',
}

df = pd.read_csv(
    RAW_PATH,
    dtype=DTYPES,
    parse_dates=['posting_date'],
    low_memory=False
)

print('=== BASELINE ===')
print(f'Shape: {df.shape}')
print(f'\nColumn dtypes:\n{df.dtypes}')
print(f'\nNull counts:\n{df.isnull().sum()}')

=== BASELINE ===
Shape: (426880, 26)

Column dtypes:
id                Int64
url              object
region           object
region_url       object
price           float64
year            float64
manufacturer     object
model            object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
VIN              object
drive            object
size             object
type             object
paint_color      object
image_url        object
description      object
county          float64
state            object
lat             float64
long            float64
posting_date     object
dtype: object

Null counts:
id                   0
url                  0
region               0
region_url           0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8

In [5]:
print('=== .describe() — BASELINE ===')
df[['price', 'year', 'odometer']].describe()

=== .describe() — BASELINE ===


,price,year,odometer
count,"426,880.00","425,675.00","422,480.00"
mean,"75,199.03","2,011.24","98,043.33"
std,"12,182,282.17",9.45,"213,881.50"
min,0.00,"1,900.00",0.00
25%,"5,900.00","2,008.00","37,704.00"
50%,"13,950.00","2,013.00","85,548.00"
75%,"26,485.75","2,017.00","133,542.50"
max,"3,736,928,711.00","2,022.00","10,000,000.00"


## 3 — Drop price outliers  `(price < 500  or  price > 150 000)`

In [6]:
before = len(df)

# Keep only rows with price in [500, 150_000]
df = df[(df['price'] >= 500) & (df['price'] <= 150_000)].copy()

dropped = before - len(df)
print(f'Dropped {dropped:,} rows with price outside [500, 150 000]')
print(f'Shape after step 3: {df.shape}')
print()
df[['price', 'odometer']].describe()

Dropped 42,290 rows with price outside [500, 150 000]
Shape after step 3: (384590, 26)



,price,odometer
count,"384,590.00","382,471.00"
mean,"19,284.19","98,531.34"
std,"14,842.55","192,014.54"
min,500.00,0.00
25%,"7,800.00","38,221.50"
50%,"15,900.00","87,700.00"
75%,"27,990.00","136,000.00"
max,"150,000.00","10,000,000.00"


## 4 — Drop odometer outliers  `(odometer > 300 000)`

In [7]:
before = len(df)

df = df[df['odometer'].isna() | (df['odometer'] <= 300_000)].copy()

dropped = before - len(df)
print(f'Dropped {dropped:,} rows with odometer > 300 000')
print(f'Shape after step 4: {df.shape}')
print()
df[['price', 'odometer']].describe()

Dropped 2,711 rows with odometer > 300 000
Shape after step 4: (381879, 26)



,price,odometer
count,"381,879.00","379,760.00"
mean,"19,312.15","91,746.45"
std,"14,818.43","61,985.25"
min,500.00,0.00
25%,"7,890.00","37,888.00"
50%,"15,966.00","87,000.00"
75%,"27,990.00","135,000.00"
max,"150,000.00","300,000.00"


## 5 — Parse `posting_date` → date features

In [8]:
# posting_date was already parsed in read_csv; coerce anything that failed
df['posting_date'] = pd.to_datetime(df['posting_date'], utc=True, errors='coerce')

# Convert to tz-naive for easier handling
df['posting_date'] = df['posting_date'].dt.tz_localize(None) \
    if df['posting_date'].dt.tz is None \
    else df['posting_date'].dt.tz_convert(None)

df['year_month']   = df['posting_date'].dt.to_period('M').astype(str)   # e.g. '2021-04'
df['month']        = df['posting_date'].dt.month                         # 1-12
df['day_of_week']  = df['posting_date'].dt.day_name()                    # 'Monday' …

n_missing_date = df['posting_date'].isna().sum()
print(f'posting_date NaT count: {n_missing_date:,}')
print(f'Shape after step 5: {df.shape}')
print()
print('Sample date features:')
df[['posting_date', 'year_month', 'month', 'day_of_week']].dropna().head(8)

posting_date NaT count: 62
Shape after step 5: (381879, 29)

Sample date features:


,posting_date,year_month,month,day_of_week
27,2021-05-04 17:31:18,2021-05,5.00,Tuesday
28,2021-05-04 17:31:08,2021-05,5.00,Tuesday
29,2021-05-04 17:31:25,2021-05,5.00,Tuesday
30,2021-05-04 15:41:31,2021-05,5.00,Tuesday
31,2021-05-03 19:02:03,2021-05,5.00,Monday
32,2021-05-03 18:41:25,2021-05,5.00,Monday
33,2021-05-03 17:41:33,2021-05,5.00,Monday
34,2021-05-03 17:12:59,2021-05,5.00,Monday


In [9]:
print('=== .describe() after date parsing ===')
df[['price', 'odometer', 'month']].describe()

=== .describe() after date parsing ===


,price,odometer,month
count,"381,879.00","379,760.00","381,817.00"
mean,"19,312.15","91,746.45",4.26
std,"14,818.43","61,985.25",0.44
min,500.00,0.00,4.00
25%,"7,890.00","37,888.00",4.00
50%,"15,966.00","87,000.00",4.00
75%,"27,990.00","135,000.00",5.00
max,"150,000.00","300,000.00",5.00


## 6 — Keep only top-20 makes by listing count

In [10]:
# Normalise manufacturer to lower-case for consistent grouping
df['manufacturer'] = df['manufacturer'].str.lower().str.strip()

top20_makes = (
    df['manufacturer']
    .value_counts()
    .head(20)
    .index
    .tolist()
)

print('Top 20 makes (by listing count):')
print(df['manufacturer'].value_counts().head(20).to_string())

before = len(df)
df = df[df['manufacturer'].isin(top20_makes)].copy()
dropped = before - len(df)

print(f'\nDropped {dropped:,} rows with non-top-20 manufacturer')
print(f'Shape after step 6: {df.shape}')

Top 20 makes (by listing count):
manufacturer
ford             63337
chevrolet        49178
toyota           30737
honda            19321
nissan           16928
jeep             16804
ram              15993
gmc              15131
bmw              13299
dodge            11672
mercedes-benz    10026
hyundai           9119
subaru            8759
volkswagen        8480
lexus             7578
kia               7304
audi              7008
cadillac          6472
acura             5543
chrysler          5520

Dropped 53,670 rows with non-top-20 manufacturer
Shape after step 6: (328209, 29)


In [11]:
print('=== .describe() after top-20 make filter ===')
df[['price', 'odometer']].describe()

=== .describe() after top-20 make filter ===


,price,odometer
count,"328,209.00","326,322.00"
mean,"19,384.68","93,665.88"
std,"14,584.20","61,847.90"
min,500.00,0.00
25%,"7,995.00","40,238.00"
50%,"15,990.00","89,417.00"
75%,"27,990.00","136,797.75"
max,"150,000.00","300,000.00"


## 7 — Add `price_per_mile` feature

In [12]:
# Guard against division by zero and NaN odometer
df['price_per_mile'] = np.where(
    df['odometer'].isna() | (df['odometer'] == 0),
    np.nan,
    df['price'] / df['odometer']
)

print(f'price_per_mile — null count: {df["price_per_mile"].isna().sum():,}')
print(f'Shape after step 7: {df.shape}')
print()
print('=== .describe() after adding price_per_mile ===')
df[['price', 'odometer', 'price_per_mile']].describe()

price_per_mile — null count: 2,689
Shape after step 7: (328209, 30)

=== .describe() after adding price_per_mile ===


,price,odometer,price_per_mile
count,"328,209.00","326,322.00","325,520.00"
mean,"19,384.68","93,665.88",83.21
std,"14,584.20","61,847.90","1,524.30"
min,500.00,0.00,0.00
25%,"7,995.00","40,238.00",0.06
50%,"15,990.00","89,417.00",0.18
75%,"27,990.00","136,797.75",0.65
max,"150,000.00","300,000.00","119,999.00"


## 8 — Summary stats dict & save to Drive

In [13]:
summary_stats = {
    # Shape
    'final_rows':         int(df.shape[0]),
    'final_cols':         int(df.shape[1]),
    # Price
    'price_mean':         round(float(df['price'].mean()), 2),
    'price_median':       round(float(df['price'].median()), 2),
    'price_std':          round(float(df['price'].std()), 2),
    'price_min':          round(float(df['price'].min()), 2),
    'price_max':          round(float(df['price'].max()), 2),
    # Odometer
    'odometer_mean':      round(float(df['odometer'].mean(skipna=True)), 2),
    'odometer_median':    round(float(df['odometer'].median(skipna=True)), 2),
    'odometer_null_pct':  round(float(df['odometer'].isna().mean() * 100), 2),
    # price_per_mile
    'ppm_mean':           round(float(df['price_per_mile'].mean(skipna=True)), 4),
    'ppm_median':         round(float(df['price_per_mile'].median(skipna=True)), 4),
    'ppm_null_pct':       round(float(df['price_per_mile'].isna().mean() * 100), 2),
    # Date coverage
    'posting_date_min':   str(df['posting_date'].min()),
    'posting_date_max':   str(df['posting_date'].max()),
    # Categorical
    'unique_makes':       int(df['manufacturer'].nunique()),
    'top_make':           str(df['manufacturer'].value_counts().idxmax()),
    'unique_states':      int(df['state'].nunique()),
}

import json
print('=== summary_stats ===')
print(json.dumps(summary_stats, indent=2))

=== summary_stats ===
{
  "final_rows": 328209,
  "final_cols": 30,
  "price_mean": 19384.68,
  "price_median": 15990.0,
  "price_std": 14584.2,
  "price_min": 500.0,
  "price_max": 150000.0,
  "odometer_mean": 93665.88,
  "odometer_median": 89417.0,
  "odometer_null_pct": 0.57,
  "ppm_mean": 83.2114,
  "ppm_median": 0.1835,
  "ppm_null_pct": 0.82,
  "posting_date_min": "2021-04-04 07:00:25",
  "posting_date_max": "2021-05-05 04:24:09",
  "unique_makes": 20,
  "top_make": "ford",
  "unique_states": 51
}


In [14]:
# Save cleaned DataFrame to Drive
df.to_csv(OUTPUT_PATH, index=False)
print(f'Saved {len(df):,} rows → {OUTPUT_PATH}')

# Also download a local copy straight to your browser
LOCAL_COPY = '/content/cleaned_cars.csv'
import shutil
shutil.copy(OUTPUT_PATH, LOCAL_COPY)

from google.colab import files
files.download(LOCAL_COPY)
print('Browser download triggered for cleaned_cars.csv')

Saved 328,209 rows → /content/drive/MyDrive/car_data/cleaned_cars.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Browser download triggered for cleaned_cars.csv


## Final — Full `.describe()` on cleaned data

In [15]:
print(f'=== FINAL shape: {df.shape} ===')
df.describe(include='all')

=== FINAL shape: (328209, 30) ===


,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date,year_month,month,day_of_week,price_per_mile
count,"328,209.00",328209,328209,328209,"328,209.00","328,209.00",328209,324546,199698,197127,326205,"326,322.00",322302,326643,202603,231258,92723,258049,233579,328209,328207,0.00,328209,"325,099.00","325,099.00",328209,328209,"328,209.00",328209,"325,520.00"
unique,<NA>,328209,404,413,NaN,NaN,20,18820,6,8,5,NaN,6,3,89953,3,4,13,12,188257,278007,NaN,51,NaN,NaN,NaN,2,NaN,7,NaN
top,<NA>,https://wyoming.craigslist.org/ctd/d/atlanta-2...,columbus,https://grandrapids.craigslist.org,NaN,NaN,ford,f-150,good,6 cylinders,gas,NaN,clean,automatic,1FMJU1JT1HEA52352,4wd,full-size,sedan,white,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Call or text today to find out more. (602) 620...,NaN,ca,NaN,NaN,NaN,2021-04,NaN,Tuesday,NaN
freq,<NA>,1,2806,2568,NaN,NaN,63337,6945,96266,74070,277420,NaN,311046,260844,261,107404,49311,64845,62001,5836,189,NaN,37586,NaN,NaN,NaN,240747,NaN,59443,NaN
mean,"7,311,532,156.90",NaN,NaN,NaN,"19,384.68","2,011.34",NaN,NaN,NaN,NaN,NaN,"93,665.88",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.57,-94.60,2021-04-24 02:05:01.349468928,NaN,4.27,NaN,83.21
min,"7,301,583,321.00",NaN,NaN,NaN,500.00,"1,900.00",NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-81.84,-159.72,2021-04-04 07:00:25,NaN,4.00,NaN,0.00
25%,"7,308,185,573.00",NaN,NaN,NaN,"7,995.00","2,008.00",NaN,NaN,NaN,NaN,NaN,"40,238.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.76,-111.62,2021-04-17 13:16:11,NaN,4.00,NaN,0.06
50%,"7,312,711,826.00",NaN,NaN,NaN,"15,990.00","2,013.00",NaN,NaN,NaN,NaN,NaN,"89,417.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.24,-88.26,2021-04-26 11:37:31,NaN,4.00,NaN,0.18
75%,"7,315,251,494.00",NaN,NaN,NaN,"27,990.00","2,017.00",NaN,NaN,NaN,NaN,NaN,"136,797.75",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.45,-81.03,2021-05-01 13:26:57,NaN,5.00,NaN,0.65
max,"7,317,101,084.00",NaN,NaN,NaN,"150,000.00","2,022.00",NaN,NaN,NaN,NaN,NaN,"300,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.39,173.89,2021-05-05 04:24:09,NaN,5.00,NaN,"119,999.00"


In [16]:
print('Sample rows from cleaned DataFrame:')
df[['manufacturer', 'year', 'price', 'odometer',
    'price_per_mile', 'year_month', 'month', 'day_of_week']].head(10)

Sample rows from cleaned DataFrame:


,manufacturer,year,price,odometer,price_per_mile,year_month,month,day_of_week
27,gmc,"2,014.00","33,590.00","57,923.00",0.58,2021-05,5.00,Tuesday
28,chevrolet,"2,010.00","22,590.00","71,229.00",0.32,2021-05,5.00,Tuesday
29,chevrolet,"2,020.00","39,590.00","19,160.00",2.07,2021-05,5.00,Tuesday
30,toyota,"2,017.00","30,990.00","41,124.00",0.75,2021-05,5.00,Tuesday
31,ford,"2,013.00","15,000.00","128,000.00",0.12,2021-05,5.00,Monday
32,gmc,"2,012.00","27,990.00","68,696.00",0.41,2021-05,5.00,Monday
33,chevrolet,"2,016.00","34,590.00","29,499.00",1.17,2021-05,5.00,Monday
34,toyota,"2,019.00","35,000.00","43,000.00",0.81,2021-05,5.00,Monday
35,chevrolet,"2,016.00","29,990.00","17,302.00",1.73,2021-05,5.00,Monday
36,chevrolet,"2,011.00","38,590.00","30,237.00",1.28,2021-05,5.00,Monday
